## Train Liveness check model

The original CASIA-FASD is not readily availabe, so I use smaller subset from Kaggle for this demo

In [1]:
import gdown
import zipfile
import os
import numpy as np
from PIL import Image

# Download dataset (Casia FASD)
file_id = '1nXoBxXhfGs5IiSojc8ZUHnlesxgXzcg7'
# file_id = '1m9dVjwU-KajHbuUMVHgENg62fUdCZg1f'
url = f'https://drive.google.com/uc?id={file_id}'

zip_file_path = '/content/casia_fasd.zip'
gdown.download(url, zip_file_path)

extract_folder_path = '/content/casia_fasd'
os.makedirs(extract_folder_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder_path)

Downloading...
From (original): https://drive.google.com/uc?id=1nXoBxXhfGs5IiSojc8ZUHnlesxgXzcg7
From (redirected): https://drive.google.com/uc?id=1nXoBxXhfGs5IiSojc8ZUHnlesxgXzcg7&confirm=t&uuid=fd16ad7b-9c0a-4d12-893d-e3e596b31752
To: /content/casia_fasd.zip
100%|██████████| 74.0M/74.0M [00:02<00:00, 25.8MB/s]


In [2]:
def calculate_normalization(folder_path):
    def load_images_from_folder(folder_path):
        images = []
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            try:
                with Image.open(img_path) as img:
                    img = img.convert('RGB')
                    images.append(np.array(img))
            except IOError:
                print(f"Error loading image: {img_path}")
        return images

    def compute_normalization_values(images):
        images_array = np.stack(images, axis=0)
        mean = np.mean(images_array, axis=(0, 1, 2))
        std = np.std(images_array, axis=(0, 1, 2))
        return mean, std

    images = load_images_from_folder(folder_path)

    mean, std = compute_normalization_values(images)

    return mean,std

# only use training data for calculating normalization value
folder_path = '/content/casia_fasd/train_img/train_img/color/'
normalization_values = calculate_normalization(folder_path)
normalization_values


(array([105.32685981,  91.9504575 ,  91.54538125]),
 array([63.06708699, 58.47346108, 58.98859229]))

In [4]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, accuracy_score


# Define the dataset class
class ImageDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None):
        self.root_dir = os.path.join(root_dir, f'{split}_img/{split}_img/color')
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for file_name in os.listdir(self.root_dir):
            if file_name.endswith('.jpg'):
                label = 'real' if 'real' in file_name else 'fake'
                self.image_paths.append(os.path.join(self.root_dir, file_name))
                self.labels.append(0 if label == 'fake' else 1)  # Encode labels as 0 and 1

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        label = torch.tensor(label, dtype=torch.float)  # Ensure labels are float
        return image, label


# Define data transformations
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=normalization_values[0], std=normalization_values[1]),
])

val_test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=normalization_values[0], std=normalization_values[1]),
])

# Load datasets
full_train_dataset = ImageDataset(root_dir='/content/casia_fasd', split='train', transform=train_transform)

train_dataset, val_dataset = random_split(full_train_dataset, [0.8, 0.2])
val_dataset.dataset.transform = val_test_transform  # Change to transform func without augmentation

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)
test_dataset = ImageDataset(root_dir='/content/casia_fasd', split='test', transform=val_test_transform)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)


# Define the model
class SpoofNet(nn.Module):
    def __init__(self):
        super(SpoofNet, self).__init__()
        # Load MobileNetV2 with pretrained weights
        self.mobilenet = mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
        self.features = self.mobilenet.features

        # Adding extra layers
        self.conv2d = nn.Conv2d(1280, 32, kernel_size=(3, 3), padding=1)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.features(x)
        x = self.conv2d(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x


# Initialize and move the model to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SpoofNet().to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()  # For binary classification
optimizer = optim.Adam(model.parameters(), lr=1e-4)
threshold = 0.5  # Apply threshold for binary classification


# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, threshold=0.5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Ensure labels are float
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0.0
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), labels)
                val_loss += loss.item() * inputs.size(0)
                preds = (outputs.squeeze() > threshold)
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        val_f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
        val_recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
        val_precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}, F1 Score: {val_f1:.4f}, Recall: {val_recall:.4f}, Precision: {val_precision:.4f}')


# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, 3, threshold)


def test_model(model, test_loader, threshold=0.5):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = (outputs.squeeze() > threshold)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    return y_true, y_pred


# Evaluate the model
y_true, y_pred = test_model(model, test_loader, threshold)

# Calculate additional metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)

# Print metrics and classification report
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')
print('Classification Report:')
print(classification_report(y_true, y_pred, target_names=['Fake', 'Real']))


Epoch 1/3, Loss: 0.2085, Validation Loss: 0.6382, F1 Score: 0.6623, Recall: 0.7644, Precision: 0.5842
Epoch 2/3, Loss: 0.0388, Validation Loss: 1.0236, F1 Score: 0.6694, Recall: 0.7674, Precision: 0.8217
Epoch 3/3, Loss: 0.0282, Validation Loss: 0.8332, F1 Score: 0.6731, Recall: 0.7644, Precision: 0.7045
Accuracy: 0.7608
F1 Score: 0.6698
Recall: 0.7608
Precision: 0.7529
Classification Report:
              precision    recall  f1-score   support

        Fake       0.76      1.00      0.86      1817
        Real       0.73      0.04      0.08       591

    accuracy                           0.76      2408
   macro avg       0.74      0.52      0.47      2408
weighted avg       0.75      0.76      0.67      2408



In [5]:
torch.save(model.state_dict(), 'spoof_weight.pth')

## Train Face recognition model

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, accuracy_score
from sklearn.datasets import fetch_lfw_people

# Load the LFW dataset
lfw_dataset = fetch_lfw_people(min_faces_per_person=70, color=True)
X = lfw_dataset.images
y = lfw_dataset.target
target_names = lfw_dataset.target_names


# Preprocess the images
class LFWDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create the full dataset
full_dataset = LFWDataset(images=X, labels=y, transform=transform)


# Use random_split to split the dataset into train, val, and test
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [0.72, 0.08, 0.2])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Define the model
class FaceRecogNet(nn.Module):
    def __init__(self, num_classes):
        super(FaceRecogNet, self).__init__()
        # Load MobileNetV2 with the weights argument
        self.mobilenet = mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
        self.mobilenet.classifier = nn.Sequential(
            nn.Linear(self.mobilenet.last_channel, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.mobilenet(x)


num_classes = len(target_names)

# Initialize and move the model to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FaceRecogNet(num_classes=num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0.0
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        val_f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
        val_recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
        val_precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}, F1 Score: {val_f1:.4f}, Recall: {val_recall:.4f}, Precision: {val_precision:.4f}')


# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, 3)


# Evaluation function
def test_model(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    return y_true, y_pred


# Evaluate the model
y_true, y_pred = test_model(model, test_loader)

# Calculate additional metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)

# Print metrics and classification report
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')
print('Classification Report:')
print(classification_report(y_true, y_pred, target_names=target_names))


Epoch 1/3, Loss: 1.6902, Validation Loss: 1.4725, F1 Score: 0.2963, Recall: 0.4660, Precision: 0.2172
Epoch 2/3, Loss: 1.3060, Validation Loss: 1.0471, F1 Score: 0.5277, Recall: 0.6602, Precision: 0.4417
Epoch 3/3, Loss: 0.9478, Validation Loss: 0.7602, F1 Score: 0.6279, Recall: 0.7087, Precision: 0.6208
Accuracy: 0.6887
F1 Score: 0.6095
Recall: 0.6887
Precision: 0.6463
Classification Report:
                   precision    recall  f1-score   support

     Ariel Sharon       1.00      0.06      0.11        18
     Colin Powell       0.49      1.00      0.66        40
  Donald Rumsfeld       0.92      0.41      0.56        27
    George W Bush       0.80      0.99      0.89       117
Gerhard Schroeder       0.00      0.00      0.00        24
      Hugo Chavez       0.00      0.00      0.00        11
       Tony Blair       0.50      0.45      0.47        20

         accuracy                           0.69       257
        macro avg       0.53      0.41      0.38       257
     weighte

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
torch.save(model.state_dict(), 'face_recognition_weight.pth')

In [10]:
import json
with open('target_names.json','w') as f:
  json.dump(list(target_names),f)